In [3]:
!pip install scikit-learn sentence-transformers umap-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 593.3 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 7.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=61935b0340b0568ac89b496942cd2842b8521f5a488318bc37140c499011f7ad
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
  Created wheel for umap-learn: filename=umap_learn-0.5.5-py3-none-any.whl size=86831 sha256=adb36876634f0a57d98316bac472752890739fd48e279cffb18afc0121beda02
  Stored in directory: /root/.cache/pip/wheels/3a/70/07/428d2b58660a1a3b431db59b806a10da736612ebbc66c1bcc5
Successfully built sentence-transformers uma

In [4]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics.cluster import normalized_mutual_info_score, adjusted_rand_score
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
from umap import UMAP
import matplotlib.pyplot as plt

In [10]:
# import train data
train_data = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
corpus_train = train_data.data[:2000]
labels_train = train_data.target[:2000]

In [14]:
train = pd.DataFrame({'Document': corpus_train, 'Category': labels_train})

In [15]:
train.head(5)

,Document,Category
0,I was wondering if anyone out there could enli...,7
1,A fair number of brave souls who upgraded thei...,4
2,"well folks, my mac plus finally gave up the gh...",4
3,\nDo you have Weitek's address/phone number? ...,1
4,"From article <C5owCB.n3p@world.std.com>, by to...",14


In [8]:
train['Category'].unique(), train['Category'].nunique()

(array([ 7,  4,  1, 14, 16, 13,  3,  2,  8, 19,  6,  0, 12,  5, 10,  9, 15,
        17, 18, 11]),
 20)

In [13]:
train.shape

(2000, 2)

In [ ]:
'''# import test data
test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))
corpus_test = test.data[:00]
labels_test = test.target[:00]
k = len(set(labels_test))'''

In [ ]:
#test = pd.DataFrame({'Document': corpus_test, 'Category': labels_test})

In [ ]:
#test.head()

In [ ]:
#test['Category'].unique(), test['Category'].nunique()

(array([ 7,  5,  0, 17, 19, 13, 15,  1,  2,  8,  4,  6, 16, 14,  3, 11,  9,
        10, 18, 12]),
 20)

In [16]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
embeddings = model.encode(corpus_train)

In [18]:
embeddings.shape

(2000, 384)

In [ ]:
#embeddings_test = model.encode(corpus_test)

In [19]:
def umap_dim_red(mat, p):
    umap_model = UMAP(n_components=p)
    red_mat = umap_model.fit_transform(mat)
    return red_mat

p = 3
red_emd = umap_dim_red(embeddings, p)